# Kubernetes

This is my compilation of tutorial [develop ML model with MLflow and deploy to Kubernetes](https://mlflow.org/docs/latest/deployment/deploy-model-to-kubernetes/tutorial.html) on the official mlflow site.

## Setup

We'll need an instance of mlflow to start the following cell in the docker.

In [1]:
%%bash
docker run -p 5000:5000 -dt --name mlflow_deploy --rm \
    ghcr.io/mlflow/mlflow \
    bash -c "mlflow server --host 0.0.0.0 --port 5000"

43656fb3ee54e5e1a7508d9c58f9e88f396e9a2bdd0ed47dc3edd684817ee8dd


We will need place where docker images can be stored - docker registry. It can be Docker Hub. But here we're going to create our local registry. It's a big fun to have docker registry in docker.

In [13]:
!docker run -d -p 5001:5000 --rm --name local-registry registry

2475d11f455e79561d53fc736d58189ff89d6a3ae0a6cda336d6edfd282bea03


In [3]:
import mlflow
import requests
import numpy as np
import subprocess
from sklearn import datasets, metrics
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split

mlflow.set_tracking_uri("http://localhost:5000")

Once you've tried everything on this notebook, it's a good idea to delete all the containers you've created.

In [19]:
%%bash
docker stop mlflow_deploy local-registry

mlflow_deploy
local-registry


## Model

### Creating model

As a model, we will use a model built on top of the Docker dataframe.

In [4]:
def eval_metrics(pred, actual):
    rmse = np.sqrt(metrics.mean_squared_error(actual, pred))
    mae = metrics.mean_absolute_error(actual, pred)
    r2 = metrics.r2_score(actual, pred)
    return rmse, mae, r2

# Set th experiment name
mlflow.set_experiment("wine-quality")

# Enable auto-logging to MLflow
mlflow.sklearn.autolog()

# Load wine quality dataset
X, y = datasets.load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Start a run and train a model
with mlflow.start_run(run_name="default-params") as run:
    run_id = run.info.run_id
    lr = ElasticNet()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)
    metr = eval_metrics(y_pred, y_test)

model_name = "wine-quality"
result = mlflow.register_model(
    f"runs:/{run_id}/model", model_name
)

2024/06/08 18:11:52 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.
2024/06/08 18:11:52 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/06/08 18:11:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/f-kobak-distance-desctop/Documents/knowledge/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'wine-quality'.
2024/06/08 18:11:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: wine-quality, version 1
Created version '1' of model 'wine-quality'.


### Check

To make sure everything's OK, let's try wrapping our model in an API locally.

In [8]:
start_api_command = f'MLFLOW_TRACKING_URI=http://localhost:5000 mlflow models serve --no-conda -m "models:/{model_name}/1" -p 4242'
ans = subprocess.run([
    'gnome-terminal', '--', 'bash', '-c', start_api_command
])

And a request to the newly created api will take shape.

In [9]:
url = "http://127.0.0.1:4242/invocations"
headers = {'Content-Type': 'application/json',}
data = {
    "inputs": [[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]]
}
response = requests.post(url, headers=headers, json=data)

print(response.text)

{"predictions": [0.39371689840943747]}


## Kubernetes

Create a Kubernetes namespace.

In [18]:
!kubectl create namespace mlflow-kserve-test

Error from server (AlreadyExists): namespaces "mlflow-kserve-test" already exists


You need to create an image for your model. The following command will preform it:

```bash
MLFLOW_TRACKING_URI=http://localhost:5000 mlflow models build-docker -m "models:/wine-quality/1" -n wine_image --enable-mlserver
```

**Note** this command takes a long time, but you only need to run it once.

You need to run the following commands to push the image you just created to the local Docker registry.
```bash
docker tag wine_image:latest localhost:5001/wine_image:latest
docker push localhost:5001/wine_image:latest
```

In [17]:
%%writefile kubernetes/config.yaml
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "mlflow-wine-classifier"
  namespace: "mlflow-kserve-test"
spec:
  predictor:
    containers:
      - name: "mlflow-wine-classifier"
        image: "localhost:5001/wine_image"
        ports:
          - containerPort: 8080
            protocol: TCP
        env:
          - name: PROTOCOL
            value: "v2"

Writing kubernetes/config.yaml


In [ ]:
!kubectl apply -f kubernetes_files/config.yaml